In [ ]:
from light_curves import LightCurve
from exposures import Calexp
from task import Run

Run(): class to process and save a bunch of light curves in the same calexps.
* For each Run() instance, all of the data is saved in a folder with the date and time name.

### Main structure

- Create a Run() instance and a list of parameters (dict).
- Add the light curves choosing the params and location (setting ra/dec or a random point within a distance from the first light curve location)
- Add a check point (log_task)

In [ ]:
process = Run()
print(process.main_path)
print(process.tasks) # Saved tasks used
params = [{"t_0": 60700.5, "t_E": 110, "u_0": 0.5, "m_base": 19.5},
          {"t_0": 60000.5, "t_E": 60, "u_0": 0.5, "m_base": 22.5}]

# for p in params:
#     process.add_lc(p, dist=0.2)

process.add_lc(params[0], ra=57.67048780800139, dec=-32.49757834967935, plot=True)

# process.add_lc(params[1], dist=0.2) 
process.add_lc(params[1], ra=57.59451632893858, dec=-32.481152201226145, plot = True)

process.log_task("Add and simulate light curves", det = len(params))
process.inj_lc # List of injected light curves

- Set and configure tasks to be used in the injection-extraction process

Iterate over the collected calexp:
- Injection
- Measure (contains the entire process with the selected approach)
- Find fluxes in source detected table (needed for this approach but others approachs might not need it)
- Compute magnitudes

In [ ]:
# Set and config tasks
process.inject_task()
schema = process.measure_task()

# Iterate on collected calexps 
for j, dataId in enumerate(process.calexp_dataIds[:10]):
    print(f" ------ CALEXP {j} ------")
    calexp = Calexp(dataId)
    injected_exp, injected_points = process.inject_calexp(calexp, save_fit=f"calexp_{j}.fit")
    if injected_exp != None:
        # Approach to measure fluxes and save them in LightCurve.data
        sources = process.measure_calexp(injected_exp, schema)
        flux, flux_err = process.find_flux(sources, injected_points, save=dataId)
        # Compute and save magnitudes in LightCurve.data
        for f, ferr, lc in zip(flux, flux_err, process.inj_lc):
            mag, mag_err = lc.add_mag(f, ferr, dataId, exposure = injected_exp)
            print(f"ra = {lc.ra}, dec = {lc.dec}")
            print("Measured ", mag, mag_err)
            print("Injected ", lc.data["mag_sim"][j])


### Outputs

* Plot final light curves and show its data (data frame)
* Save the lc as csv